## Preparing datas

### Get data

In [1]:
# Import from Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
!cp /content/drive/MyDrive/assamese_text_data/captions_csv.csv.zip .
!cp /content/drive/MyDrive/assamese_text_data/captions_csv2.csv.zip .

In [20]:
!unzip /content/captions_csv.csv.zip
!unzip /content/captions_csv2.csv.zip

Archive:  /content/captions_csv.csv.zip
replace captions_csv.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: captions_csv.csv        
Archive:  /content/captions_csv2.csv.zip
  inflating: captions_csv2.csv       


In [5]:
import pandas as pd

In [24]:
data = pd.read_csv("/content/captions_csv.csv")
data = data['Caption']
data = data.dropna()

data2 = pd.read_csv("/content/captions_csv2.csv")
data2 = data2['wHaT dAy Is It Even #stayhomeclub']
data2 = data2.dropna()

In [25]:
data.head()

1                                                 bye 
2    Ok, a few more... sorry I just had so much fun...
3    This was one of my favorite shoots I’ve ever d...
4                 Wrapped round my finger like a ring 
5    Yesterday was so much fun!! has been such a hu...
Name: Caption, dtype: object

In [26]:
data2.head()

0    Vitamin C for your fashion diet #KatyPursey #S...
1    do you know the hotter the fire the purer the ...
2                                     @ryanseacrest 👍🏻
3             working hard or hardly working idk 🤷🏼‍♀️
4    Step into your destiny with The Bondie 🔮✨ #Sho...
Name: wHaT dAy Is It Even #stayhomeclub, dtype: object

In [ ]:
# data_merged = ''
# for index in range(len(data)):
#   data_merged += data.iloc[index]

# print(data_merged)

In [35]:
print(type(data), type(data2))
print(len(data), len(data2))

<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>
17100 14411


In [36]:
# data3 = pd.merge(data, data2, right_index=True, left_index=True)
data3 = data + data2
data3 = data3.squeeze()
print(type(data3))
print(data3.head())
print(len(data3))

<class 'pandas.core.series.Series'>
0                                                  NaN
1    bye do you know the hotter the fire the purer ...
2    Ok, a few more... sorry I just had so much fun...
3    This was one of my favorite shoots I’ve ever d...
4    Wrapped round my finger like a ring Step into ...
dtype: object
19415


### Tokenisation

In [8]:
# Standard Data Science Libraries
import pickle
import math
import pandas as pd
import numpy as np
from numpy import array

# Neural Net Preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Neural Net Layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

# Neural Net Training
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

from pickle import load

In [9]:
max_words = 50000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)

# Flatten the list of lists resulting from the tokenization. This will reduce the list
# to one dimension, allowing us to apply the sliding window technique to predict the next word
text = [item for sublist in sequences for item in sublist]
vocab_size = len(tokenizer.word_index)

In [10]:
print(vocab_size, sequences[:10])
print(text[:5])

13574 [[610], [984, 8, 282, 58, 409, 3, 35, 86, 16, 50, 109, 23, 34], [10, 26, 48, 11, 2, 126, 1652, 304, 136, 394, 27, 5, 8, 913, 3156, 478, 7, 537, 2, 125, 6, 27, 5, 4, 124, 62, 1291, 8, 3766, 7, 1, 676, 3157], [1653, 677, 2, 1792, 51, 8, 1066], [332, 26, 16, 50, 109, 105, 79, 181, 8, 872, 2148, 9, 2, 53, 3, 15, 67, 3767, 67, 1535, 6, 67, 201, 155, 24, 1, 676, 216, 88, 6649, 179, 67, 1, 382, 49], [312], [137, 940, 24, 3, 26, 8, 479, 3768, 2729, 16, 2413, 7, 5, 4, 1962, 92, 985, 2, 329, 18, 701, 6650, 27, 5, 16, 50, 7, 250, 355, 16, 1292, 4, 14, 25, 8, 416, 538, 121, 805, 20, 3, 626, 3769, 17, 7, 5, 20, 4, 806, 1, 320, 6, 1293, 1, 15, 6651], [3, 313, 5, 63, 29, 4742, 1, 329, 3, 35, 139, 4, 27, 5, 63, 7, 1017, 14, 6, 10, 232, 11, 191, 18, 1018, 1, 116, 4, 14, 5, 33, 101, 1112, 409, 107, 137, 79, 8, 421, 3770, 2414, 95, 192, 39, 2, 1793, 320, 175, 4743], [194, 18, 236, 1367, 1793], [725, 8, 725, 359]]
[610, 984, 8, 282, 58]


In [11]:
# Training on 19 words to predict the 20th
sentence_len = 20
pred_len = 1
train_len = sentence_len - pred_len
seq = []
# Sliding window to generate train data
for i in range(len(text)-sentence_len):
    seq.append(text[i:i+sentence_len])
# Reverse dictionary to decode tokenized sequences back to words
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

# Each row in seq is a 20 word long window. We append he first 19 words as the input to predict the 20th word
trainX = []
trainy = []
for i in seq:
    trainX.append(i[:train_len])
    trainy.append(i[-1])

In [12]:
print(trainX[:5], trainy[:5])
print(reverse_word_map[trainX[0][0]])

[[610, 984, 8, 282, 58, 409, 3, 35, 86, 16, 50, 109, 23, 34, 10, 26, 48, 11, 2], [984, 8, 282, 58, 409, 3, 35, 86, 16, 50, 109, 23, 34, 10, 26, 48, 11, 2, 126], [8, 282, 58, 409, 3, 35, 86, 16, 50, 109, 23, 34, 10, 26, 48, 11, 2, 126, 1652], [282, 58, 409, 3, 35, 86, 16, 50, 109, 23, 34, 10, 26, 48, 11, 2, 126, 1652, 304], [58, 409, 3, 35, 86, 16, 50, 109, 23, 34, 10, 26, 48, 11, 2, 126, 1652, 304, 136]] [126, 1652, 304, 136, 394]
bye


## Model Architecture

In [14]:
# Model
model = Sequential([
    Embedding(vocab_size+1, 50, input_length=train_len),
    LSTM(100, return_sequences=True),
    LSTM(100),
    Dense(100, activation='relu'),
    Dropout(0.1),
    Dense(vocab_size, activation='softmax')
])

# Train model with checkpoints
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
filepath = './model_weights_english_v1.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callback_list = [checkpoint]
history = model.fit(np.asarray(trainX),
          pd.get_dummies(np.asarray(trainy)),
          epochs = 50,
          batch_size = 128,
          callbacks = callback_list,
          verbose = 1)

Epoch 1/50
1256/1256 [==============================] - ETA: 0s - loss: 7.1513 - accuracy: 0.0342
Epoch 1: loss improved from inf to 7.15135, saving model to ./model_weights_v1.hdf5
1256/1256 [==============================] - 57s 37ms/step - loss: 7.1513 - accuracy: 0.0342
Epoch 2/50
1256/1256 [==============================] - ETA: 0s - loss: 6.7721 - accuracy: 0.0522
Epoch 2: loss improved from 7.15135 to 6.77211, saving model to ./model_weights_v1.hdf5
1256/1256 [==============================] - 19s 15ms/step - loss: 6.7721 - accuracy: 0.0522
Epoch 3/50
1256/1256 [==============================] - ETA: 0s - loss: 6.5359 - accuracy: 0.0694
Epoch 3: loss improved from 6.77211 to 6.53595, saving model to ./model_weights_v1.hdf5
1256/1256 [==============================] - 19s 15ms/step - loss: 6.5359 - accuracy: 0.0694
Epoch 4/50
1252/1256 [============================>.] - ETA: 0s - loss: 6.3472 - accuracy: 0.0809
Epoch 4: loss improved from 6.53595 to 6.34676, saving model to ./mod

In [15]:
def gen(model,seq,max_len = 20):
    ''' Generates a sequence given a string seq using specified model until the total sequence length
    reaches max_len'''
    # Tokenize the input string
    tokenized_sent = tokenizer.texts_to_sequences([seq])
    max_len = max_len+len(tokenized_sent[0])
    # If sentence is not as long as the desired sentence length, we need to 'pad sequence' so that
    # the array input shape is correct going into our LSTM. the `pad_sequences` function adds 
    # zeroes to the left side of our sequence until it becomes 19 long, the number of input features.
    while len(tokenized_sent[0]) < max_len:
        padded_sentence = pad_sequences(tokenized_sent[-19:],maxlen=19)
        op = model.predict(np.asarray(padded_sentence).reshape(1,-1))
        tokenized_sent[0].append(op.argmax()+1)
        
    return " ".join(map(lambda x : reverse_word_map[x],tokenized_sent[0]))

In [16]:
a = 'chilling in the beach'
res = gen(model, a)
print(res)

1/1 [==============================] - 0s 19ms/step
chilling in the beach curling rant curled up by hair flashbackfriday luscious coat arabia round dolce yzy szn 3 coat dress shop up in


In [17]:
# save reverse word map
import json

json_object = json.dumps(reverse_word_map) 
print(json_object)

with open("reverse_word_map_english.json", "w") as file:
  file.write(json_object)

{"1": "the", "2": "my", "3": "i", "4": "to", "5": "you", "6": "and", "7": "for", "8": "a", "9": "in", "10": "this", "11": "of", "12": "with", "13": "on", "14": "me", "15": "love", "16": "so", "17": "is", "18": "it", "19": "at", "20": "all", "21": "new", "22": "we", "23": "that", "24": "out", "25": "be", "26": "was", "27": "thank", "28": "com", "29": "are", "30": "happy", "31": "up", "32": "by", "33": "have", "34": "day", "35": "just", "36": "now", "37": "your", "38": "i'm", "39": "from", "40": "our", "41": "but", "42": "birthday", "43": "night", "44": "what", "45": "today", "46": "look", "47": "best", "48": "one", "49": "time", "50": "much", "51": "like", "52": "get", "53": "life", "54": "see", "55": "do", "56": "last", "57": "go", "58": "more", "59": "when", "60": "collection", "61": "know", "62": "who", "63": "guys", "64": "these", "65": "baby", "66": "how", "67": "her", "68": "back", "69": "make", "70": "u", "71": "good", "72": "not", "73": "little", "74": "will", "75": "tonight", "

In [18]:
# saving tokenizer
with open('tokenizer_english.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# load using
# with open('tokenizer.pickle', 'rb') as handle:
#     tokenizer = pickle.load(handle)